In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## 1. Graph

In [ ]:
import networkx as nx

number_of_nodes = 3
erdos_prob = .6

graph = nx.erdos_renyi_graph(number_of_nodes, erdos_prob)
adjacency = nx.to_numpy_matrix(graph).A

In [ ]:
adjacency

## 2.1. Dymanic Formulation

In [ ]:
def biochemical(data, t):
    F = .5
    B = .2
    R = 3
    
    n = len(data)
    result = np.empty(n)
    
    for i in range(0,n):
        sigma = 0
        for j in range (0,n):
            sigma = sigma + R*(adjacency[i,j])*data[i]*data[j]
        result[i] = F - B*data[i] - sigma
    
    return result

In [ ]:
def birthdead(data, t):
    B = .02
    R = .3
    a = 1
    b = 1
    
    n = len(data)
    result = np.empty(n)
    
    for i in range(0,n):
        sigma = 0
        for j in range (0,n):
            sigma = sigma + R*(adjacency[i,j])*pow(data[j],a)
        result[i] = -B*pow(data[i],b) + sigma
    
    return result

In [ ]:
def epidemic(data, t):
    B = .02
    R = .3
    b = 1
    
    n = len(data)
    result = np.empty(n)
    
    for i in range(0,n):
        sigma = 0
        for j in range (0,n):
            sigma = sigma + R*(adjacency[i,j])*(1-data[i])*data[j]
        result[i] = -B*pow(data[i],b) + sigma
    
    return result

## 2.2. Generate Time-series

In [ ]:
from scipy.integrate import odeint

dt = .002
t_train = np.arange(0, 10, dt)
x0_train = np.random.rand(number_of_nodes)
x_train = odeint(biochemical, x0_train, t_train)

## 3. Fit SINDy

In [ ]:
import pysindy as ps

stlsq_optimizer = ps.STLSQ(threshold=.01, alpha=.5)

model = ps.SINDy(optimizer=stlsq_optimizer)
model.fit(x_train, t=dt)
model.print()

In [ ]:
t_test = np.arange(0, 15, dt)
x0_test = np.empty(number_of_nodes)
x0_test.fill(.2)
x_test = odeint(biochemical, x0_test, t_test)  

# Compare SINDy-predicted derivatives with finite difference derivatives
print('Model score: %f' % model.score(x_test, t=dt))